In [27]:
import urllib.request
import pandas as pd
import numpy as np
from typing import Dict, List

In [2]:
url = 'https://www.questrade.com/docs/librariesprovider7/default-document-library/questrade_bonds_list_excel.xlsx'
headers = {'User-Agent': 'My User Agent 1.0'}
req = urllib.request.Request(url, headers=headers)
with urllib.request.urlopen(req) as r:
    xl = r.read()

xl_data = pd.read_excel(xl, sheet_name=None)

In [11]:
xl_data.keys()

dict_keys(['GIC LESS THEN 1 YEAR', 'GIC 1-6 YEAR', 'MUNICIPAL', 'COUPONS', 'CORPORATE', 'HIGH YIELD', 'PROVINCES'])

In [133]:
xl_data['GIC 1-6 YEAR'].head(10)

,TERM DEPOSITS / GIC: 1 - 6 YEARS,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,Today:,2023-06-12 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Settlement:,2023-06-12 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,*Interest Rates are annual. Prices and quantit...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,GICs may be purchased from 9:00AM to 1:00PM E...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Please be advised that GICs cannot be redeemed...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Community Trust Company is a member of the Que...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,TERM DEPOSITS/GICS: 1 TO 6 YEARS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Financial Institution,Compound Frequency,Payment Frequency,Redeemable,Redeemable Term,Minimum Deposit,Term in Years,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,1,2,3,4,5,6


In [134]:
df_names = {
    'GIC LESS THEN 1 YEAR': 'gic_1y_less',
    'GIC 1-6 YEAR': 'gic_1y_more',
    'MUNICIPAL': 'municipal',
    'COUPONS': 'coupon',
    'CORPORATE': 'corporate',
    'HIGH YIELD': 'high_yield',
    'PROVINCES': 'provinces',
    }

In [135]:
def get_tbl(xl_dict:Dict[str, pd.DataFrame], names_map:Dict[str,str])->Dict[str, pd.DataFrame]:
    'Loop though `xl_dict` and format the dfs'
    dict_out = {}
    for sh, df in xl_dict.items():
        rows_to_drop = []
        for row in df.itertuples():
            if row[1] == 'Today:':
                today = row[2]
                rows_to_drop.append(row[0])
            elif row[1] == 'Settlement:':
                settle_date = row[2]
                rows_to_drop.append(row[0])
            elif row[2] is np.NAN and row[3] is np.NAN: 
                rows_to_drop.append(row[0])
        df_out = df.drop(rows_to_drop, axis=0)
        df_out.columns = df_out.iloc[0]
        df_out = df_out.iloc[1:]
        dict_out[names_map[sh]] = df_out
        df_out['quote_date'] = today
        df_out['settle_date'] = settle_date
    return dict_out

In [215]:
out = get_tbl(xl_data, df_names)

In [216]:
out['gic_1y_more'].head(3)

8,Financial Institution,Compound Frequency,Payment Frequency,Redeemable,Redeemable Term,Minimum Deposit,Term in Years,NaN,NaN,NaN,NaN,NaN,quote_date,settle_date
10,B2B Bank,A,E,no,-,5000,5.12%,5.07%,4.85%,4.62%,4.62%,-,2023-06-12,2023-06-12
11,NaN,N,A,no,-,5000,5.12%,5.07%,4.85%,4.62%,4.62%,-,2023-06-12,2023-06-12
12,Community Trust,A,E,no,-,5000,5.08%,5.03%,4.60%,4.20%,4.25%,-,2023-06-12,2023-06-12


In [217]:
def fix_dfs(dict_df:Dict[str,pd.DataFrame])->pd.DataFrame:
    for name, df in dict_df.items():
        if name == "gic_1y_less":
            df.columns = ["Financial Institution", "Redeemable", "Minimum Deposit", "30_days", "60_days", "90_days",
                          "120_days", "180_days", "270_days", "quote_date", "settle_date"]
            col_mask = [c for c in df.columns if c.endswith("_days")]
            df[col_mask] = df[col_mask].apply(lambda c: c.str.replace("%", ""))
            df[col_mask] = df[col_mask].apply(lambda c: pd.to_numeric(c, errors="coerce"))
            df['Financial Institution'] = df['Financial Institution'].ffill()
        elif name == "gic_1y_more":
            df.columns = ["Financial Institution", "Compound Frequency", "Payment Frequency", "Redeemable", 
                          "Redeemable Term", "Minimum Deposit", "1_y", "2_y", "3_y",
                          "4_y", "5_y", "6_y", "quote_date", "settle_date"]
            col_mask = [c for c in df.columns if c.endswith("_y")]
            df[col_mask] = df[col_mask].apply(lambda c: c.str.replace("%", ""))
            df[col_mask] = df[col_mask].apply(lambda c: pd.to_numeric(c, errors="coerce"))
            df['Financial Institution'] = df['Financial Institution'].ffill()
            df = df.loc[:df[df['Financial Institution'] == "Heading"].index.values.item()-1]
    return dict_df

In [218]:
out_fixed = fix_dfs(out)
out1 = out_fixed['gic_1y_more']
out1

,Financial Institution,Compound Frequency,Payment Frequency,Redeemable,Redeemable Term,Minimum Deposit,1_y,2_y,3_y,4_y,5_y,6_y,quote_date,settle_date
10,B2B Bank,A,E,no,-,5000,5.12,5.07,4.85,4.62,4.62,NaN,2023-06-12,2023-06-12
11,B2B Bank,N,A,no,-,5000,5.12,5.07,4.85,4.62,4.62,NaN,2023-06-12,2023-06-12
12,Community Trust,A,E,no,-,5000,5.08,5.03,4.60,4.20,4.25,NaN,2023-06-12,2023-06-12
13,Community Trust,A,E,no,-,50000,5.18,5.13,4.70,4.30,4.35,NaN,2023-06-12,2023-06-12
14,Community Trust,N,A,no,-,5000,5.08,5.03,4.60,4.20,4.25,NaN,2023-06-12,2023-06-12
15,Community Trust,N,A,no,-,50000,5.18,5.13,4.70,4.30,4.35,NaN,2023-06-12,2023-06-12
16,Community Trust.,A,E,no,-,5000,5.18,5.13,4.70,4.30,4.35,NaN,2023-06-12,2023-06-12
17,Community Trust.,N,A,no,-,5000,5.18,5.13,4.70,4.30,4.35,NaN,2023-06-12,2023-06-12
18,Effort Trust,A,E,no,-,5000,5.12,4.43,3.90,3.80,3.70,NaN,2023-06-12,2023-06-12
19,Effort Trust,N,A,no,-,5000,5.12,4.43,3.90,3.80,3.70,NaN,2023-06-12,2023-06-12
